# 第七章：函数
 使用 def 语句定义函数是所有程序的基础。
本章的目标是讲解一些更加高级和不常见的函数定义与使用模式。
涉及到的内容包括默认参数、任意数量参数、强制关键字参数、注解和闭包。
另外，一些高级的控制流和利用回调函数传递数据的技术在这里也会讲解到。

In [ ]:
def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)

def print_result(result):
    print('Got:', result)
def add(x, y):
    return x + y
apply_async(add, (2, 3), callback=print_result)

apply_async(add, ('hello', 'world'), callback=print_result)


为了让回调函数访问外部信息，一种方法是使用一个绑定方法来代替一个简单函数。

class ResultHandler:

    def __init__(self):
        self.sequence = 0

    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))

r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler)

第二种方式，作为类的替代，可以使用一个闭包捕获状态值，
还有另外一个更高级的方法，可以使用协程来完成同样的事情：

def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))

对于协程，你需要使用它的 send() 方法作为回调函数，如下所示：

handler = make_handler()
next(handler) # Advance to the yield
apply_async(add, (2, 3), callback=handler.send)

apply_async(add, ('hello', 'world'), callback=handler.send)

## 7.11 内联回调函数


### 问题


当你编写使用回调函数的代码的时候，担心很多小函数的扩张可能会弄乱程序控制流。
你希望找到某个方法来让代码看上去更像是一个普通的执行序列。

### 解决方案


通过使用生成器和协程可以使得回调函数内联在某个函数中。
为了演示说明，假设你有如下所示的一个执行某种计算任务然后调用一个回调函数的函数(参考7.10小节)：

接下来让我们看一下下面的代码，它包含了一个 Async 类和一个 inlined_async 装饰器：

In [2]:
from queue import Queue
from functools import wraps

def apply_async(func, args, *, callback):
    result = func(*args)
    callback(result)

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args

def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper



def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(10):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')
test()

5
helloworld
0
2
4
6
8
10
12
14
16
18
Goodbye


你会发现，除了那个特别的装饰器和 yield 语句外，其他地方并没有出现任何的回调函数(其实是在后台定义的)。

### 讨论


本小节会实实在在的测试你关于回调函数、生成器和控制流的知识。

首先，在需要使用到回调的代码中，关键点在于当前计算工作会挂起并在将来的某个时候重启(比如异步执行)。
当计算重启时，回调函数被调用来继续处理结果。apply_async() 函数演示了执行回调的实际逻辑，
尽管实际情况中它可能会更加复杂(包括线程、进程、事件处理器等等)。

计算的暂停与重启思路跟生成器函数的执行模型不谋而合。
具体来讲，yield 操作会使一个生成器函数产生一个值并暂停。
接下来调用生成器的 __next__() 或 send() 方法又会让它从暂停处继续执行。

根据这个思路，这一小节的核心就在 inline_async() 装饰器函数中了。
关键点就是，装饰器会逐步遍历生成器函数的所有 yield 语句，每一次一个。
为了这样做，刚开始的时候创建了一个 result 队列并向里面放入一个 None 值。
然后开始一个循环操作，从队列中取出结果值并发送给生成器，它会持续到下一个 yield 语句，
在这里一个 Async 的实例被接受到。然后循环开始检查函数和参数，并开始进行异步计算 apply_async() 。
然而，这个计算有个最诡异部分是它并没有使用一个普通的回调函数，而是用队列的 put() 方法来回调。

这时候，是时候详细解释下到底发生了什么了。主循环立即返回顶部并在队列上执行 get() 操作。
如果数据存在，它一定是 put() 回调存放的结果。如果没有数据，那么先暂停操作并等待结果的到来。
这个具体怎样实现是由 apply_async() 函数来决定的。
如果你不相信会有这么神奇的事情，你可以使用 multiprocessing 库来试一下，
在单独的进程中执行异步计算操作，如下所示：

In [ ]:
if __name__ == '__main__':
    import multiprocessing
    pool = multiprocessing.Pool()
    apply_async = pool.apply_async

    # Run the test function
    test()

实际上你会发现这个真的就是这样的，但是要解释清楚具体的控制流得需要点时间了。

将复杂的控制流隐藏到生成器函数背后的例子在标准库和第三方包中都能看到。
比如，在 contextlib 中的 @contextmanager 装饰器使用了一个令人费解的技巧，
通过一个 yield 语句将进入和离开上下文管理器粘合在一起。
另外非常流行的 Twisted 包中也包含了非常类似的内联回调。

## 7.12 访问闭包中定义的变量


### 问题


你想要扩展函数中的某个闭包，允许它能访问和修改函数的内部变量。

### 解决方案


通常来讲，闭包的内部变量对于外界来讲是完全隐藏的。
但是，你可以通过编写访问函数并将其作为函数属性绑定到闭包上来实现这个目的。例如：

In [ ]:
def sample():
    n = 0
    # Closure function
    def func():
        print('n=', n)

    # Accessor methods for n
    def get_n():
        return n

    def set_n(value):
        nonlocal n
        n = value

    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func

下面是使用的例子:

In [ ]:
f = sample()
f()

In [ ]:
f.set_n(10)
f()

In [ ]:
f.get_n()

### 讨论


为了说明清楚它如何工作的，有两点需要解释一下。首先，nonlocal 声明可以让我们编写函数来修改内部变量的值。
其次，函数属性允许我们用一种很简单的方式将访问方法绑定到闭包函数上，这个跟实例方法很像(尽管并没有定义任何类)。

还可以进一步的扩展，让闭包模拟类的实例。你要做的仅仅是复制上面的内部函数到一个字典实例中并返回它即可。例如：

In [ ]:
import sys
class ClosureInstance:
    def __init__(self, locals=None):
        if locals is None:
            locals = sys._getframe(1).f_locals

        # Update instance dictionary with callables
        self.__dict__.update((key,value) for key, value in locals.items()
                            if callable(value) )
    # Redirect special methods
    def __len__(self):
        return self.__dict__['__len__']()

# Example use
def Stack():
    items = []
    def push(item):
        items.append(item)

    def pop():
        return items.pop()

    def __len__():
        return len(items)

    return ClosureInstance()

下面是一个交互式会话来演示它是如何工作的：

In [ ]:
s = Stack()
s

In [ ]:
s.push(10)
s.push(20)
s.push('Hello')
len(s)

In [ ]:
s.pop()

In [ ]:
s.pop()

In [ ]:
s.pop()

有趣的是，这个代码运行起来会比一个普通的类定义要快很多。你可能会像下面这样测试它跟一个类的性能对比：

In [ ]:
class Stack2:
    def __init__(self):
        self.items = []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def __len__(self):
        return len(self.items)

如果这样做，你会得到类似如下的结果：

In [ ]:
from timeit import timeit
# Test involving closures
s = Stack()
timeit('s.push(1);s.pop()', 'from __main__ import s')

In [ ]:
# Test involving a class
s = Stack2()
timeit('s.push(1);s.pop()', 'from __main__ import s')

结果显示，闭包的方案运行起来要快大概8%，大部分原因是因为对实例变量的简化访问，
闭包更快是因为不会涉及到额外的self变量。

Raymond Hettinger对于这个问题设计出了更加难以理解的改进方案。不过，你得考虑下是否真的需要在你代码中这样做，
而且它只是真实类的一个奇怪的替换而已，例如，类的主要特性如继承、属性、描述器或类方法都是不能用的。
并且你要做一些其他的工作才能让一些特殊方法生效(比如上面 ClosureInstance 中重写过的 __len__() 实现。)

最后，你可能还会让其他阅读你代码的人感到疑惑，为什么它看起来不像一个普通的类定义呢？
(当然，他们也想知道为什么它运行起来会更快)。尽管如此，这对于怎样访问闭包的内部变量也不失为一个有趣的例子。

总体上讲，在配置的时候给闭包添加方法会有更多的实用功能，
比如你需要重置内部状态、刷新缓冲区、清除缓存或其他的反馈机制的时候。